# Experiment 2
- Load the saved model and replace only the output layer of the model (to align it to the new problem).
- Train and evaluate the model (for 50 epochs) on the cats and dogs dataset.
### Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

### Load Pretrained model

In [ ]:
model = keras.models.load_model("experiment1_model.keras")

### Change the output layer
Go from multi class classification to binary classification.  

In [ ]:
# Remove the last layer and freeze the base model
base_model = keras.Model(inputs=model.input, outputs=model.layers[-2].output)
base_model.trainable = False  # Freeze pretrained layers

# Add new output layer (Binary Classification)
new_output = layers.Dense(1, activation="sigmoid")(base_model.output)

# Create a new model
new_model = keras.Model(inputs=base_model.input, outputs=new_output)

### Compile the model

In [ ]:
new_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)

### Create dataset

In [7]:
import tensorflow_datasets as tfds

# Load the dataset
dataset_name = "cats_vs_dogs"
image_size = (180, 180)
batch_size = 128
seed = 1337

def preprocess(image, label):
    image = tf.image.resize(image, image_size)
    image = image
    return image, label

train_ds, val_ds = tfds.load(dataset_name, split=["train[:80%]", "train[80%:]"], as_supervised=True)
train_ds = train_ds.map(preprocess).batch(batch_size).shuffle(seed)
val_ds = val_ds.map(preprocess).batch(batch_size)

Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 124.95 url/s]


KeyError: "There is no item named 'PetImages\\\\Cat\\\\0.jpg' in the archive"

### Train

In [ ]:
epochs = 50
callbacks = [
    keras.callbacks.ModelCheckpoint("models/experiment2_epoch_{epoch}.keras"),
]

new_model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=callbacks
)

new_model.save("experiment2_model.keras")